# Analyzing a Curves output file with `pyña`

Let's do a quick refresher about Curves output. First, Curves output for a trajectory is just the Curves output for each frame, concatenated together. That is, if you had `frame1.out`, `frame2.out`, and `frame3.out`, then `cat frame1.out frame2.out frame3.out` would be the same as the analysis for the whole trajectory.

Within each frame, there are 5 groups of data. Following the Curves labels, we call these groups A-E (although perhaps it would be better to give them more descriptive names?). They include the following:

* `groupA` : Base pair axis parameters (`xdisp`, `ydisp`, `inclin`, `tip`, `ax-bend`)
* `groupB` : Intra-base pair parameters (`shear`, `stretch`, `stagger`, `buckle`, `propel`, `opening`)
* `groupC` : Inter-base pair parameters (`shift`, `slide`, `rise`, `tilt`, `roll`, `twist`, `h-ris`, `h-twi`)
* `groupD` : Backbone parameters (not yet supported)
* `groupE` : Groove paramemeters (`w12`, `d12`, `w21`, `d21`)

Note that, while Curves capitalizes the first letter of each measurement, `pyña` does not.

Getting your data into a `pyña` object is as easy as the following two lines:

In [1]:
import pyna
curves = pyna.CurvesAnalysis('./data/shorter.data')

## Using `pandas.Panel`s for each group's data

The data from each group of parameters is put into a `pandas.Panel` object. By using `pandas`, we have many powerful options for analysis and export of data. However, I've simplified a few common tasks to have more familiar names within `pyña`.

In [2]:
curves.panels['groupB']

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 4 (major_axis) x 30 (minor_axis)
Items axis: buckle to stretch
Major_axis axis: 0 to 3
Minor_axis axis: 1.0 to 30.0

The `Panel` object includes 3 axes. In `pyña`, the "item" axis corresponds to the property being measured (e.g., `buckle`), the "minor" axis corresponds to the location within the strand as labeled by Curves (usually base-pair number), and the "major" axis corresponds to time.

If we want to get a table with all the buckle angles, with each column representing a different base pair and each row representing a different frame of the trajectory, it is as easy as this:

In [3]:
curves.panels['groupB']['buckle']

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,...,21.0,22.0,23.0,24.0,25.0,26.0,27.0,28.0,29.0,30.0
0,-0.7,0.1,-5.6,-8.6,-4.7,-1.3,-4.1,-2.5,3.5,-1.9,...,-3.3,-11.9,2.8,-1.2,-15.0,-10.6,2.1,-13.3,-14.4,-14.8
1,-2.1,4.2,-0.2,-4.6,7.8,19.2,1.3,2.5,-8.3,-8.6,...,-2.9,8.5,3.2,-11.2,14.0,15.2,1.2,-5.1,0.9,-9.4
2,12.5,5.9,1.9,-11.1,-20.9,-2.1,-2.1,2.5,11.1,-10.3,...,-5.6,25.5,1.6,-11.8,1.0,10.3,9.5,-2.5,6.8,-8.5
3,-7.4,3.3,-3.5,-15.2,-1.6,-4.9,-13.6,-15.2,-10.5,11.6,...,0.9,12.4,3.0,-4.6,-7.5,5.1,-9.3,-9.4,-16.1,-11.9


Note how this dataframe differs from the Curves output: each column from Curves is now a separate table; what were the rows in Curves are columns for `pyña`. And for `pyña` each row represented a time, whereas Curves has each time in a separate table. At the end of this document, I'll show how to reconstruct the Curves approach.

The main object to generate statistics on subsets of that data is `pyna.StrandStatistics`. You create a `StrandStatistics` object with a dataframe, 0 or more `locations`, and 0 or more `times`. The locations refer to the columns in the `pyña` dataframe, and the times refer to its rows.

If you don't set `locations`, then the statistics are generated for all locations. The same is true of `times`.

So if you want to see the average buckle angle, averaged over all time and all base pairs, you can obtain that with:

In [4]:
pyna.StrandStatistics(curves.panels['groupB']['buckle']).mean()

-1.3375000000000001

Of course, it may be better to save the statistics object. The function `.summary()` shows a number of properties (each available as a function of that name, with no arguments). Internally, the structure has a property `.df`, to which any pandas statistics can be applied, as well as `.np`, which returns a numpy array for use with numpy or scipy statistical analysis.

In [5]:
all_buckle = pyna.StrandStatistics(curves.panels['groupB']['buckle'])
print all_buckle.summary()

mean: -1.3375
std:  9.43724061454
min:  -23.3
max:  26.0



To get information about the distribution of the buckle angle for base pair 6 (distribution over time), do the following:


In [6]:
bp6_byTime = pyna.StrandStatistics(curves.panels['groupB']['buckle'], locations=6)
print bp6_byTime.summary()

mean: 2.725
std:  9.60530452406
min:  -4.9
max:  19.2



To get information about the distribution of buckle angles in a given frame (averaging over base pair number), you can do this:

In [7]:
frame0 = pyna.StrandStatistics(curves.panels['groupB']['buckle'], times=0)
print frame0.summary()

mean: -3.29
std:  7.27765300996
min:  -15.0
max:  16.6



You can also limit that to a subset of the base pairs by using the column headers for them:

In [8]:
frame0_subset_columns = pyna.StrandStatistics(curves.panels['groupB']['buckle'], times=0, locations=[4,5,6,7])
print frame0_subset_columns.summary()

mean: -4.675
std:  2.60420333308
min:  -8.6
max:  -1.3



Say you want the distribution of buckle angles for a group of base pairs averaged over both that group and over all frames:

In [9]:
sub_columns = pyna.StrandStatistics(curves.panels['groupB']['buckle'], locations=[4,5,6,7])
print sub_columns.summary()

mean: -2.48125
std:  8.94498593836
min:  -20.9
max:  19.2



Or perhaps you only want to look at that for a subsection of the frames:

In [10]:
subtime_subcol = pyna.StrandStatistics(curves.panels['groupB']['buckle'], times=[0,2], locations=[4, 5, 6, 7])
print subtime_subcol.summary()

mean: -4.4
std:  6.55591336123
min:  -20.9
max:  2.5



Note also that `pyña` correctly handles the groove parameters, which can't always be calculated for all base pair lables. Where there is no answer, `pyña` returns "not-a-number" (NaN).

In [11]:
curves.panels['groupE']['w12']

,1.5,2.0,2.5,3.0,3.5,4.0,4.5,5.0,5.5,6.0,...,25.0,25.5,26.0,26.5,27.0,27.5,28.0,28.5,29.0,29.5
0,NaN,NaN,NaN,7.0,6.4,6.0,6.1,6.0,5.5,5.5,...,5.7,7.1,8.3,7.2,5.6,4.8,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,6.1,6.6,6.9,7.1,6.6,6.1,...,6.9,5.9,4.8,4.3,4.1,4.7,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,4.5,3.5,3.5,5.0,6.3,6.3,6.0,...,7.1,7.8,7.7,6.6,5.2,4.1,4.3,NaN,NaN,NaN
3,NaN,NaN,NaN,6.6,5.6,4.9,5.1,5.2,4.8,5.1,...,4.5,5.3,6.4,5.9,5.1,5.1,NaN,NaN,NaN,NaN


Having NaNs still allows correct statistics:

In [12]:
wstat3 = pyna.StrandStatistics(curves.panels['groupE']['w12'], locations=3)
print wstat3.summary()

mean: 6.03333333333
std:  1.09645894689
min:  4.5
max:  7.0



### Co-keys

There are several columns in the Curves output which remain unchanged from frame to frame. These help give additional information about what the Curves base pair key (the row in Curves, the column in `pyña`'s dataframes). Since they are unchanged with each frame, we only save them in one place, called `co_keys`. For example, to see the co-key for the buckle (part of group B) in the column labeled "6", you type:

In [13]:
curves.co_keys['groupB'][6]

['T', '7-A', '26']

These are the same columns you would see in Curves, following the "6)" label.

### Curves-like panels

Maybe you want to double-check `pyña`'s output against the output in Curves, or maybe you just like the paradigm used by Curves more. Here's how to create `Panels` using the Curves convention, where the "item" axis is time, the "minor" axis is the measurement name, and the "major" axis is the base pair label:

In [14]:
# TODO

You still have one panel per group, but the organization of each panel has changed.

In [15]:
# TODO: 
# curves_style['groupE'][0]